In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

files=[]

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        files.append(os.path.join(dirname, filename))
        print(files[-1])

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv(files[-1], parse_dates=['date'])

In [ ]:
def weird_division(n, d):
    return n / d if d else 0

df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)

df.drop(['total_revenue' , 'measurable_impressions'], axis = 1, inplace=True)
df['day_week'] = df['date'].dt.dayofweek



In [ ]:
test = df[df['date']>'2019-06-22']
train = df[df['date']<='2019-06-22']

In [ ]:
test = test[test['CPM']>=0]

train = train[train['CPM'] < np.percentile(test['CPM'].values, 95)]
test = test[test['CPM']<np.percentile(test['CPM'].values, 95)]

In [ ]:
y_train = train['CPM']
X_train = train.drop(columns='CPM')
y_test = test['CPM']
X_test = test.drop(columns='CPM')

In [ ]:
category_field = df.nunique()[df.nunique()<=10].index.to_list()

cat = CatBoostRegressor(loss_function = 'RMSE', cat_features=category_field)

cat.fit(X_train,y_train,verbose=False, plot=True)

In [ ]:
y_pred = cat.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred)